# Распределение товаров по категориям

## Инициализация

In [ ]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

In [ ]:
import pandas as pd
import numpy as np

import catalog.runner
from catalog.models import *

## Загружаю данные о продуктах

### Описание полей:
- **id** - идентификатор в базе;
- **vendor** - идентификатор производителя;
- **category** - категория продукта (идентификатор);
- **unit** - единица измерения;
- **double** - если продукт является дублем имеющегося - ссылка на исходный;
- **price_type** - тип цены (как правило - розничная);
- **currency** - идентификатор валюты цены (на данный момент только рубль);
- **name** - имя продукта;
- **article** - артикул продукта (уникален для производителя);
- **alias** - производитель, артикул и имя продукта, приведённые в вид, оптимизированный для индексации поиска;
- **description** - описание продукта (реализация в будущем);
- **edited** - редактировался ли продукт вручную;
- **tested** - объект проверен вручную, все данные о нём верны;
- **for_export** - экспортировать ли информацию во внешние системы, например, в Яндекс.Маркет (реализация в будущем);
- **on_stock** - количество, доступное на складах;
- **on_transit** - количество, доступное в транзитах;
- **on_factory** - количество, доступное на заказ;
- **price** - цена;
- **fixed** - фиксирована ли цена (True для некоторых программных продуктов или для акционного продукта);
- **state** - статус объекта (если False - объект не используется ни в каких обработках);
- **created** - дата и время создания объёкта в базе;
- **modified** - дата последнего изменения объекта.

In [ ]:
products = Product.objects.get_all_df()
print(products.info())
products.head()

## Загружаем все возможные наименования продуктов

У разных поставщиков один и тот-же продукт может называться по разному.

### Описание полей:
- **id** - идентификатор в базе;
- **product** - продукт, которому соответствует имя;
- **name** - имя;
- **state** - статус;
- **created** - дата и время создания;
- **modified** - дата и время последнего редактирования.

In [ ]:
input_names = ProductInputName.objects.get_all_df()
print(input_names.info())
input_names.head()

## Загружаем все возможные наименования категорий продуктов

Категории у разных поставщиков называются по разному.

### Описание полей:
- **id** - идентификатор в базе;
- **product** - продукт, которому соответствует имя категории;
- **category** - имя категори;
- **state** - статус;
- **created** - дата и время создания;
- **modified** - дата и время последнего редактирования.

In [ ]:
input_categories = ProductInputCategory.objects.get_all_df()
print(input_categories.info())
input_categories.head()

## Переносим имена продуктов и категорий в DF продуктов

In [ ]:
def get_input_names(product):
    filtered_names = input_names[input_names['product'] == product]
    names = []
    for i, row in filtered_names.iterrows():
        names.append(row['name'])
    names = ' '.join(names)
    return names

products['input_names'] = products['id'].map(get_input_names)
print(products.info())

In [ ]:
def get_input_categories(product):
    filtered_categories = input_categories[input_categories['product'] == product]
    categories = []
    for i, row in filtered_categories.iterrows():
        if row['category']:
            categories.append(row['category'])
    categories = ' '.join(categories)
    return categories

products['input_categories'] = products['id'].map(get_input_categories)
print(products.info())

In [ ]:
products.head()

## Извлечение признаков

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Вычисляем признаки из текущих имён
names_td_idf = TfidfVectorizer()
X_names = names_td_idf.fit_transform(products['name'])

print('Размер исходной мартицы:', products.shape)
print('Размер матрицы признаков:', X_names.shape)

In [ ]:
# Вычисляем признаки из исходных имён
input_names_td_idf = TfidfVectorizer()
X_input_names = input_names_td_idf.fit_transform(products['input_names'])

print('Размер исходной мартицы:', products.shape)
print('Размер матрицы признаков:', X_input_names.shape)

In [ ]:
# Вычисляем признаки из исходных категорий
input_categories_td_idf = TfidfVectorizer()
X_input_categories = input_categories_td_idf.fit_transform(products['input_categories'])

print('Размер исходной мартицы:', products.shape)
print('Размер матрицы признаков:', X_input_categories.shape)

## Кластеризация продуктов

Необходимо кластеризовать продукты, чтобы кластеры "раскидать" по категориям.

Перевести и разобраться в описании методов [sklearn.cluster](http://scikit-learn.org/stable/modules/clustering.html#clustering)

In [ ]:
# Affinity Propagation Clustering

from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs

type(X_names)